# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He; psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770352397690                   -0.53    8.0    167ms
  2   -2.771684875513       -2.88       -1.31    1.0   91.7ms
  3   -2.771714209873       -4.53       -2.53    1.0   91.4ms
  4   -2.771714664700       -6.34       -3.43    1.0   94.6ms
  5   -2.771714713982       -7.31       -3.98    2.0    113ms
  6   -2.771714715214       -8.91       -5.29    1.0   96.4ms
  7   -2.771714715249      -10.47       -5.42    2.0    116ms
  8   -2.771714715250      -11.98       -5.94    1.0    102ms
  9   -2.771714715250      -13.20       -7.23    1.0    112ms
 10   -2.771714715250      -14.01       -7.37    2.0    130ms
 11   -2.771714715250      -14.15       -7.91    1.0    104ms
 12   -2.771714715250      -13.86       -7.61    1.0    115ms
 13   -2.771714715250   +  -13.64       -8.15    1.0    114ms


-0.00013457335850779328

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770367268371                   -0.53    9.0    2.02s
  2   -2.771771964393       -2.85       -1.30    1.0    148ms
  3   -2.771801517436       -4.53       -2.68    1.0   92.7ms
  4   -2.771802073186       -6.26       -4.07    2.0    117ms
  5   -2.771802074425       -8.91       -4.95    1.0    102ms
  6   -2.771802074475      -10.30       -5.57    2.0    120ms
  7   -2.771802074476      -12.05       -6.06    1.0    102ms
  8   -2.771802074476      -13.15       -6.87    1.0    102ms
  9   -2.771802074476      -14.65       -6.96    2.0    136ms
 10   -2.771802074476      -13.81       -7.84    1.0    112ms
 11   -2.771802074476   +  -14.21       -8.54    2.0    117ms


0.017612221445125142

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457335850779328
Displaced dipole:  0.017612221445125142
Polarizability :   1.7746794803632935


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920789756e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551483364e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852767052900e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694603653596e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088789167237e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.295265061800e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 1.296181081485e-12
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 1.296181081485e-12
[ Info: GMRES linsolve in iter 2; step 1: normres = 1.080060911685e-10
[ Info: GMRES linsolve in iter 2; step 2: normres = 1.609719286048e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 3.448581793603e-13
[ Info: GMRES linsolve in iter 2; finished at step 3: normres = 3.448581793603e-13
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ *  norm of residual = 3.4484678873994404e-13
└ *  number of opera

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.